<a href="https://colab.research.google.com/github/StockRecursiveOverflow/ML/blob/main/c11_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [21]:
# First model A predict 8 classes Trouser, Dress, Coat,
# Sandal, Shirt, Sneaker, Bag, Ankle boot

#Second model B only prediction 2 classes T-shirt/top, Pullover

import numpy as np
pullover_id = 2
t_shirt_id = 0

def split_dataset(X, y):
  eight_images_id = [1,3,4,5,6,7,8,9]
  two_images = (y == pullover_id) | (y == t_shirt_id)
  y_eight_images = y[~two_images]
  y_two_images = (y[two_images] == pullover_id).astype(np.float32) # change labels to 0 or 1 for model A and B
  for eight_original_id, eight_new_id in zip(eight_images_id, range(8)):
    y_eight_images[y_eight_images == eight_original_id] = eight_new_id # change labels to 0~7 for model B_base
  return ((X[~two_images], y_eight_images), (X[two_images], y_two_images))

X_train, y_train = x_train[:-5000], y_train[:-5000]
X_valid, y_valid = x_train[-5000:], y_train[-5000:]

In [22]:
(X_train_eight, y_train_eight), (X_train_two, y_train_two) = split_dataset(X_train, y_train)
(X_valid_eight, y_valid_eight), (X_valid_two, y_valid_two) = split_dataset(X_valid, y_valid)
(X_test_eight, y_test_eight), (X_test_two, y_test_two) = split_dataset(x_test, y_test)

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_A.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_two, y_train_two, epochs=20,
                      validation_data=(X_valid_two, y_valid_two))
model_A.evaluate(X_test_two, y_test_two)

Epoch 1/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8546 - loss: 28.5521 - val_accuracy: 0.5059 - val_loss: 3.6005
Epoch 2/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9513 - loss: 0.1775 - val_accuracy: 0.5059 - val_loss: 3.1751
Epoch 3/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9574 - loss: 0.1446 - val_accuracy: 0.5020 - val_loss: 3.2032
Epoch 4/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9606 - loss: 0.1271 - val_accuracy: 0.4980 - val_loss: 3.3679
Epoch 5/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9626 - loss: 0.1190 - val_accuracy: 0.5020 - val_loss: 3.1450
Epoch 6/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9648 - loss: 0.1060 - val_accuracy: 0.4980 - val_loss: 3.3792
Epoch 7/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9650 - loss: 0.1045 - val_accuracy: 0.4922 - val_loss: 3.5162
Epoch 8/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9667 - loss: 0.0999 - val_accuracy: 0

[0.1636199653148651, 0.9549999833106995]

In [23]:
model_B_base = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_B_base.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B_base.fit(X_train_eight, y_train_eight, epochs=20,
                      validation_data=(X_valid_eight, y_valid_eight))

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6266 - loss: 29.9659 - val_accuracy: 0.1244 - val_loss: 11.5544
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8041 - loss: 0.5228 - val_accuracy: 0.1239 - val_loss: 11.8402
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8408 - loss: 0.4294 - val_accuracy: 0.1256 - val_loss: 12.0815
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8580 - loss: 0.3855 - val_accuracy: 0.1251 - val_loss: 12.4074
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8691 - loss: 0.3590 - val_accuracy: 0.1244 - val_loss: 12.4799
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8766 - loss: 0.3394 - val_accuracy: 0.1244 - val_loss: 12.6639
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8815 - loss: 0.3244 - val_accuracy: 0.1261 - val_loss: 12.7100
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8864 - loss:

In [24]:
model_B_base_clone = tf.keras.models.clone_model(model_B_base)
model_B_base_clone.set_weights(model_B_base.get_weights())
model_B = tf.keras.Sequential(model_B_base_clone.layers[:-1])
model_B.add(tf.keras.layers.Dense(1, activation="sigmoid"))

for layer in model_B.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B.compile(loss="binary_crossentropy", optimizer=optimizer,
                metrics=["accuracy"])

history = model_B.fit(X_train_two, y_train_two, epochs=4,
                      validation_data=(X_valid_two, y_valid_two))

for layer in model_B.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B.fit(X_train_two, y_train_two, epochs=16,
                           validation_data=(X_valid_two, y_valid_two))

Epoch 1/4
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8212 - loss: 0.5468 - val_accuracy: 0.4922 - val_loss: 3.0633
Epoch 2/4
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9105 - loss: 0.2525 - val_accuracy: 0.4892 - val_loss: 2.8366
Epoch 3/4
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9118 - loss: 0.2294 - val_accuracy: 0.4882 - val_loss: 2.8147
Epoch 4/4
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9136 - loss: 0.2260 - val_accuracy: 0.4863 - val_loss: 2.8211
Epoch 1/16
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9285 - loss: 0.1893 - val_accuracy: 0.5029 - val_loss: 3.1745
Epoch 2/16
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9509 - loss: 0.1341 - val_accuracy: 0.5000 - val_loss: 3.2439
Epoch 3/16
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9554 - loss: 0.1214 - val_accuracy: 0.4990 - val_loss: 3.2664
Epoch 4/16
344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9560 - loss: 0.1145 - val_accuracy: 0.4980

In [25]:
model_B.evaluate(X_test_two, y_test_two)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9628 - loss: 0.1107


[0.11492341011762619, 0.9639999866485596]